In [0]:
#functions(F)->id used for math, aggregations, transformations
#Window->time series calculation
from pyspark.sql import functions as F
from pyspark.sql.window import Window


In [0]:
# retrieving the silver tables and storing them in the form of dataframes
stock_df=spark.table("stock_prices_clean")
investor_df=spark.table("investor_master_clean")
txn_df=spark.table("portfolio_transactions_clean")

## **GOLD ANALYTICS – STOCK RETURNS**

#### **Daily Returns**

In [0]:
# grouping the data stock-wise and ordering them by date 
window_spec = Window.partitionBy("symbol").orderBy("trade_date")

stock_returns_df = stock_df.withColumn(
    "daily_return",
    (F.col("close_price") - F.lag("close_price").over(window_spec)) /
    F.lag("close_price").over(window_spec)
)


#### **Volatility (Risk Metric)**

In [0]:
#groups the stocks by symbol and calculates the standard deviation of daily returns 
stock_volatility_df = stock_returns_df.groupBy("symbol") \
    .agg(
        F.stddev("daily_return").alias("volatility")
    )


In [0]:
stock_volatility_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_stock_volatility")

In [0]:
%sql
SELECT * FROM gold_portfolio_value LIMIT 10;

trade_date,portfolio_value
2022-01-03,57686.399999999994
2022-01-04,8623.86
2022-01-06,31358.7
2022-01-10,52921.95999999999
2022-01-11,134541.01
2022-01-13,7300.599999999999
2022-01-17,60824.4
2022-01-18,null
2022-01-20,60152.4
2022-01-21,53858.88


## **GOLD ANALYTICS – PORTFOLIO PERFORMANCE**

#### **Join Transactions with Prices**

In [0]:
portfolio_base_df = txn_df.join(
    stock_df.select("symbol", "trade_date", "close_price"),
    ["symbol", "trade_date"],
    "inner"
)

#### **Position Value**

In [0]:
portfolio_positions_df = portfolio_base_df.withColumn(
    "position_value",
    F.col("quantity") * F.col("close_price")
)

#### **Daily Portfolio Value**

In [0]:
portfolio_value_df = portfolio_positions_df.groupBy("trade_date") \
    .agg(
        F.sum("position_value").alias("portfolio_value")
    ) \
    .orderBy("trade_date")

In [0]:
portfolio_value_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_portfolio_value")


In [0]:
%sql
SELECT * FROM gold_portfolio_value LIMIT 10;


trade_date,portfolio_value
2022-01-03,57686.399999999994
2022-01-04,8623.86
2022-01-06,31358.7
2022-01-10,52921.95999999999
2022-01-11,134541.01
2022-01-13,7300.599999999999
2022-01-17,60824.4
2022-01-18,null
2022-01-20,60152.4
2022-01-21,53858.88


## **GOLD ANALYTICS – SECTOR PERFORMANCE**

In [0]:
sector_kpi_df = stock_df.groupBy("sector") \
    .agg(
        F.sum("volume").alias("total_volume"),
        F.avg("close_price").alias("avg_close_price")
    )


In [0]:
sector_kpi_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_sector_kpis")

In [0]:
%sql
select * from gold_sector_kpis LIMIT 10;

sector,total_volume,avg_close_price
Telecom,2.58381647E8,297.6717113402059
Retail,2.48908944E8,162.84385714285705
Energy,2.5034565E8,309.9992900608519
Manufacturing,2.42555106E8,417.95985743380834
Technology,2.54453136E8,403.14660569105695
Automobile,2.4772384E8,290.5534888438135
Finance,2.47695748E8,324.8683265306119
Pharma,2.36183233E8,453.6865082644629
Consumer,2.50812774E8,437.3993951612903
Healthcare,2.46842115E8,149.01075203252023


## **Gold Layer: Risk-Adjusted Return**

#### **Average Return per Stock**

In [0]:
avg_return_df = stock_returns_df.groupBy("symbol") \
    .agg(
        F.avg("daily_return").alias("avg_daily_return")
    )


#### **Join with Volatility**

In [0]:
risk_adjusted_df = avg_return_df.join(
    stock_volatility_df,
    "symbol"
)


#### **Compute Risk-Adjusted Score**

In [0]:
risk_adjusted_df = risk_adjusted_df.withColumn(
    "risk_adjusted_return",
    F.col("avg_daily_return") / F.col("volatility")
)


In [0]:
risk_adjusted_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold_risk_adjusted_returns")

In [0]:
%sql
select * from gold_risk_adjusted_returns LIMIT 10;

symbol,avg_daily_return,volatility,risk_adjusted_return
ALPH,9.011752172119902E-4,0.011303457478161291,0.07972562545160142
AUTO,-1.411095631312413E-4,0.01614206990316185,-0.008741726679278057
BNKX,1.1591826569761949E-4,0.014549621264721287,0.007967098496143571
CONS,4.505973587685943E-5,0.009837613499591462,0.004580352326174502
ENGY,-5.303693509360594E-4,0.015576935811294537,-0.034048374941077875
HLTH,-0.001272342356304497,0.030701616516660014,-0.04144219427710165
METL,8.564105000399447E-4,0.011325749892917533,0.07561622922430013
PHRM,1.967982041191458E-4,0.010161365685102243,0.01936729867006707
RETL,0.001432745920763472,0.029041281499127918,0.049334803658939635
TELC,-4.176669224933199E-4,0.01504905381903335,-0.02775369983494072


## **Investor Trading Behavior Analysis**

In [0]:
from pyspark.sql.functions import count, when, col
# Calculate trading frequency
investor_trade_count = (
    txn_df
    .groupBy("investor_id")
    .agg(count("*").alias("trade_count"))
)



In [0]:

# Classify investor behavior
gold_investor_trading_behavior = (
    investor_trade_count
    .withColumn(
        "trading_behavior",
        when(col("trade_count") <= 3, "Long-Term Investor")
        .when(col("trade_count") <= 10, "Swing Trader")
        .otherwise("Frequent Trader")
    )
)


In [0]:
investor_master_df = spark.table("investor_master_clean")
gold_investor_trading_behavior = (
    gold_investor_trading_behavior
    .join(
        investor_master_df.select("investor_id", "region"),
        on="investor_id",
        how="left"
    )
)


In [0]:
gold_investor_trading_behavior.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("gold_investor_trading_behavior")


In [0]:
%sql
SELECT * from gold_investor_trading_behavior LIMIT 15;

investor_id,trade_count,trading_behavior,region
INV009,1,Long-Term Investor,US
INV155,2,Long-Term Investor,Europe
INV061,3,Long-Term Investor,Europe
INV052,2,Long-Term Investor,Asia
INV135,1,Long-Term Investor,Asia
INV289,5,Swing Trader,US
INV115,2,Long-Term Investor,Europe
INV147,4,Swing Trader,US
INV133,1,Long-Term Investor,US
INV007,3,Long-Term Investor,US


In [0]:
%sql
SHOW TABLES;


database,tableName,isTemporary
default,gold_investor_trading_behavior,false
default,gold_portfolio_value,false
default,gold_risk_adjusted_returns,false
default,gold_sector_kpis,false
default,gold_stock_volatility,false
default,investor_master_clean,false
default,portfolio_transactions_clean,false
default,stock_prices_clean,false


## Inserting a record to test manual refresh behaviour 

In [0]:
%sql
INSERT INTO gold_investor_trading_behavior
VALUES (
    9999,
    15,
    'Frequent Trader',
    'North America'
);


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT count(*) from gold_investor_trading_behavior


count(*)
274
